## Determining the (silenced) alerts that fire most often during upgrades

### High-level plan
- Query observatorium-mst for `sre:slo:upgradeoperator_upgrade_result == 0`
  - for each _id as CUUID:
    - Query SL-DB for when most recent upgrade started&ended for this $CUUID
    - Query telemeter-lts for value of alerts{_id=$CUUID} during upgrade timeframe
      - Add cluster's alerts to histogram

### ~~Querying Observatorium-MST for UUIDs of clusters that paged during their last upgrade~~

In [ ]:
from datetime import datetime, timedelta
from settings import OBSERVATORIUM_URL, OBSERVATORIUM_AUTH_COOKIE
from urllib.parse import quote
import requests

In [ ]:
# range_start = int((datetime.now() - timedelta(days=60)).timestamp())
# range_end = int(datetime.now().timestamp())
# observatorium_query = f"{OBSERVATORIUM_URL}/query_range?start={range_start}&end={range_end}&step=3600&query={quote('sre:slo:upgradeoperator_upgrade_result == 0')}"
# observatorium_results = requests.get(observatorium_query, cookies=OBSERVATORIUM_AUTH_COOKIE).json()
# if observatorium_results['status'] != "success":
#     raise ValueError("Observatorium query unsuccessful: " + str(observatorium_results))
# alerting_upgrade_cluster_uuids = set(r['metric']['_id'] for r in observatorium_results['data']['result'])
# len(alerting_upgrade_cluster_uuids)

Instead of the above approach, let's try using a pre-made list of cluster UUIDs for which an upgrade occurred in the last 90 days. Note that not all clusters in the list still exist

In [ ]:
with open("scratch/clusters_upgraded_since_2023-08-02.txt") as f:
    alerting_upgrade_cluster_uuids = set(l.strip() for l in f if len(l) > 10)
len(alerting_upgrade_cluster_uuids)

Now that we have UUIDs for clusters that paged during their last upgrade, we'll try...
### Querying OCM service log API for upgrade time windows 

In [ ]:
import re
from datetime import timezone
from util import OCMClient

In [ ]:
ocm_client = OCMClient()
version_regex = re.compile(r"version '?([-\w\.]+)['\.]")

upgrade_window_dicts = []
for cuuid in alerting_upgrade_cluster_uuids:
    # Fetch upgrade-related service logs for this cluster
    sldb_cuuid = "cluster_uuid=" + quote(cuuid)
    sldb_search = "search=" + quote("summary = 'Upgrade maintenance beginning' or summary = 'Upgrade maintenance completed'")
    sldb_response = ocm_client.get("/api/service_logs/v1/clusters/cluster_logs?" + "&".join([sldb_cuuid, sldb_search])).json()

    # Iterate over fetched service logs to determine latest upgrade start/end times and version
    upgrade_start = datetime.min.replace(tzinfo=timezone.utc)
    upgrade_end = datetime.min.replace(tzinfo=timezone.utc)
    upgrade_version = None
    for sl in sldb_response['items']:
        sl_timestamp = datetime.fromisoformat(sl['timestamp'].replace("Z", "+00:00"))
        if sl['summary'] == "Upgrade maintenance beginning" and sl_timestamp > upgrade_start:
            upgrade_start = sl_timestamp

        if sl['summary'] == "Upgrade maintenance completed" and sl_timestamp > upgrade_end:
            upgrade_end = sl_timestamp
            upgrade_version = version_regex.search(sl['description']).group(1)
    
    #print(f"Start: {upgrade_start} | End: {upgrade_end} | Version: {upgrade_version}")
    window_dict = {
        'uuid': cuuid,
        'upgrade_version': upgrade_version,
        'upgrade_start': upgrade_start,
        'upgrade_end': upgrade_end
    }
    upgrade_window_dicts.append(window_dict)
len(upgrade_window_dicts)

Now that we have clusters UUIDs and upgrade start/end times, we'll try...

### Querying Telemeter for alerts that fired during upgrades

In [ ]:
from settings import TELEMETER_URL, TELEMETER_AUTH_COOKIE, TELEMETER_CA_BUNDLE_PATH, IGNORED_ALERTS

In [ ]:
alert_dicts = []
for uwd in upgrade_window_dicts:
    range_start = int(uwd['upgrade_start'].timestamp())
    range_end = int(uwd['upgrade_end'].timestamp())
    promql = "alerts{_id=\"" + uwd['uuid'] + "\"}"
    telemeter_query = f"{TELEMETER_URL}/query_range?query={quote(promql)}&start={range_start}&end={range_end}&step=3600" 
    try:
        telemeter_results = requests.get(telemeter_query, cookies=TELEMETER_AUTH_COOKIE, verify=TELEMETER_CA_BUNDLE_PATH).json()
        if telemeter_results['status'] != "success":
            raise ValueError("Telemeter query unsuccessful: " + str(telemeter_results))
    except ValueError as ex:
                print(f"WARN: Couldn't process alert due to JSONDecodeError: {ex}")
    for tr in telemeter_results['data']['result']:
        if tr['metric']['alertname'] not in IGNORED_ALERTS:
            try:
                alert_dicts.append({
                    'name': tr['metric']['alertname'],
                    'severity': tr['metric']['severity'],
                    'namespace': tr['metric']['namespace'],
                    'upgrade_version': uwd['upgrade_version'],
                    'upgrade_timestamp': uwd['upgrade_start'],
                    'cluster_uuid': uwd['uuid']
                })
            except KeyError as ex:
                print(f"WARN: Couldn't process alert due to missing {ex}: {tr['metric']}")
len(alert_dicts)

Now that we have a list of every instance of an alert firing during a failed upgrade, we'll try...
### Analyzing alerts with NumPy

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
alerts_df = pd.DataFrame(alert_dicts).drop_duplicates(ignore_index=True)
alerts_df

In [ ]:
histogram = alerts_df.value_counts(subset=["name", "severity", "namespace"]).to_frame().reset_index()
pd.set_option('display.max_rows', 800)
histogram[histogram['count'] > 5]

In [ ]:
", ".join(IGNORED_ALERTS)